# Test destriper
Use check for a simple case the left and the right hand sides of the iterative calibration equation

In [1]:
import rings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import healpy as hp

/global/project/projectdirs/planck/software/zonca/dev/pycfitsio/__init__.py:7: UserWarning: Cannot import read_map and read_mask if healpy is not installed
  warnings.warn('Cannot import read_map and read_mask if healpy is not installed')


In [2]:
%matplotlib inline

In [3]:
config = dict(
    dipole_constraint=False,
    )

In [4]:
R = rings.RingSetManager("LFI28M", 128, tag="full",
                         del_psi=True,
                         ringsets_folder="/global/scratch2/sd/planck/user/zonca/data/ringsets_%s" % "dx11_delta",
                         fixfactor=1e3)

In [5]:
R.data = R.data.ix[:6000]
R.pids = R.data.index.unique()

In [6]:
baselines = pd.Series(np.arange(len(R.pids)), index=R.pids)

We assume to have a perfect knowledge of `m_prev` and `g_prev`, basically our solution at the previous step was without error

In [7]:
hit_map = R.create_hit_map(R.data.index)

In [8]:
assert np.isnan(hit_map).sum() == 0

In [9]:
assert np.allclose(hit_map, R.data.hits.groupby(R.data.pix).sum())

In [10]:
R.data.c = 1

In [11]:
bin_map = R.create_bin_map(R.data.c)

In [12]:
assert bin_map.I.mean() == 1

In [13]:
assert np.isnan(bin_map.I).sum() == 0

In [14]:
R.data.c = baselines.reindex(R.data.index)

In [15]:
bin_map,destriped_map,solution_baselines = R.destripe(R.data.c, maxiter=50, return_baselines=True)

In [20]:
assert np.allclose(solution_baselines - solution_baselines[0], baselines)